# 8-qubit experiments

In [1]:
import sys
sys.path.append('../src')

import numpy as np

from third_meeting.noisy_8_qubit import get_circuits_and_outputs

from utils import simulate_with_noise_3D
from utils import DistanceVisualizer
from utils import plot_3d

print("All imports successful!")

All imports successful!


In [2]:
# circuits_and_outputs = get_circuits_and_outputs()
# names = circuits_and_outputs['names']
# true_circuits = circuits_and_outputs['true_circuits']
# true_dists = circuits_and_outputs['true_dists']
# mirrored_circuits = circuits_and_outputs['mirrored_circuits']
# true_outputs = circuits_and_outputs['true_outputs']

# noise_levels = np.linspace(0, 0.02, 21)

# # Create ranges for each segment
# ranges = [
#     np.arange(10, 1000, 20),      # Usually big std reduction occurs here so we want to most precision
#     np.arange(1000, 2000, 50),    # Usually completely converges in this range
#     np.arange(2000, 3001, 100)
# ]
# shot_counts = np.concatenate(ranges)

# Simulate:
1-12:  different n_local configurations [ry, rz]
13-18: different real_amplitudes configurations
19:23: random circuits with increasing depth

In [3]:
# # Simulate circuits 0 to 23 (there might be problems with circuit 12)
# for circuit_index in range(0, 24):
#     mirrored_circuit = mirrored_circuits[circuit_index]
#     true_output = true_outputs[circuit_index]

#     true_circuit = true_circuits[circuit_index]
#     true_dist = true_dists[circuit_index]

#     hellinger_data = simulate_with_noise_3D(
#              true_circuit, true_dist, noise_levels=noise_levels, shot_counts=shot_counts, distance_type='hellinger'
#     )

#     hamming_data = []
#     for i in range(10):
#         print(f"Simulation run {i+1}/10 for circuit index {circuit_index}")
#         H = simulate_with_noise_3D(
#             mirrored_circuit, true_output, noise_levels=noise_levels, shot_counts=shot_counts, distance_type='hamming'
#         )
#         hamming_data.append(H)

#     np.savez_compressed(
#         f'../data/third_meeting/circuit_{circuit_index}_data.npz',
#         name=circuits_and_outputs['names'][circuit_index],
#         shots_array=shot_counts,
#         noise_intensities=noise_levels,
#         hamming_data=hamming_data,
#         hellinger_data=hellinger_data
#     )
    

In [4]:
# # Load all circuit data and put it into one file

# shot_counts = []
# noise_levels = []
# names_dataset = []
# hamming_dataset = []
# hellinger_dataset = []

# # Load QFT
# loaded = np.load(f'../data/third_meeting/circuit_0_data.npz', allow_pickle=True)
# shot_counts = loaded['shots_array']
# noise_levels = loaded['noise_intensities']

# names_dataset.append(loaded['name'].item())
# hamming_dataset.append(loaded['hamming_data'])
# hellinger_dataset.append(loaded['hellinger_data'])

# # Load circuits 1 to 23
# for circuit_index in range(1, 24):
#     loaded = np.load(f'../data/third_meeting/circuit_{circuit_index}_data.npz', allow_pickle=True)
#     names_dataset.append(loaded['name'].item())
#     hamming_dataset.append(loaded['hamming_data'])
#     hellinger_dataset.append(loaded['hellinger_data'])

# # Save all data into one file
# np.savez_compressed(
#     f'../data/third_meeting/all_circuits_data.npz',
#     names=names_dataset,
#     shots_array=shot_counts,
#     noise_intensities=noise_levels,
#     hamming_data=hamming_dataset,
#     hellinger_data=hellinger_dataset
# )

# Visualization

In [6]:
# Load all circuits and visualize
loaded = np.load(f'../data/third_meeting/all_circuits_data.npz', allow_pickle=True)
names_dataset = loaded['names']
shot_counts = loaded['shots_array']
noise_levels = loaded['noise_intensities']
hamming_dataset = loaded['hamming_data']
hellinger_dataset = loaded['hellinger_data']

for i in range(len(names_dataset)):
    visualizer = DistanceVisualizer(shot_counts, noise_levels, names_dataset[i], hamming_dataset[i], hellinger_dataset[i], 8)
    visualizer.create_dashboard()

Visualizing data for circuit: QFT


interactive(children=(FloatSlider(value=0.01, description='Noise Intensity:', max=0.02, step=0.000952380952380…

Visualizing data for circuit: varQC_linear_2r


interactive(children=(FloatSlider(value=0.01, description='Noise Intensity:', max=0.02, step=0.000952380952380…

Visualizing data for circuit: varQC_linear_4r


...

interactive(children=(FloatSlider(value=0.01, description='Noise Intensity:', max=0.02, step=0.000952380952380…

Visualizing data for circuit: RandomCircuit_depth40


interactive(children=(FloatSlider(value=0.01, description='Noise Intensity:', max=0.02, step=0.000952380952380…

Visualizing data for circuit: RandomCircuit_depth80


interactive(children=(FloatSlider(value=0.01, description='Noise Intensity:', max=0.02, step=0.000952380952380…